# Machine Learning Engineer

## Introduction: Welcome to the Machine Learning Engineering Role!

This course will walk you through everything you need to know to be a Rockstar
ML Engineer. Unlike other courses, the DS Role courses will present some
assignments in the form of iPython Notebooks. It is highly recommended to use
Colab to complete these assignments.

### Course Outline
- ML Model Interfaces
- Model Tuning & Pipelines
- Clustering Model & Lookup Tables
- Model Serialization

### Random Monsters: MonsterLab

In [1]:
!pip install MonsterLab

  Using cached MonsterLab-1.2.2-py3-none-any.whl (4.4 kB)


In [2]:
import joblib
from typing import List, Tuple

import pandas as pd
from pandas import DataFrame
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

from MonsterLab import Monster

Before we can do machine learning we need some data!

### Random Monster

In [3]:
Monster()

Name: Steam Devil
Type: Devilkin
Level: 7
Rarity: Rank 0
Damage: 7d2+2
Health: 14.0
Energy: 14.53
Sanity: 13.77
Time Stamp: 2022-03-29 13:10:29

Generate Mock Monster Data. 5000 should make for a good model, but play with it, see what you can find with different values for the `number` variable below.

In [4]:
number = 5000
df = pd.DataFrame(Monster().to_dict() for _ in range(number))
df.to_csv("monsters.csv", index=False)
df

,Name,Type,Level,Rarity,Damage,Health,Energy,Sanity,Time Stamp
0,Red Dragon,Dragon,9,Rank 0,9d2+1,17.84,17.90,17.59,2022-03-29 13:10:47
1,Djinni,Elemental,6,Rank 1,6d4+4,23.17,22.87,23.56,2022-03-29 13:10:47
2,Ghoul,Undead,7,Rank 2,7d6+2,43.64,44.76,43.42,2022-03-29 13:10:47
3,Imp,Demonic,4,Rank 3,4d8+2,34.08,29.55,34.87,2022-03-29 13:10:47
4,Kobold Villager,Devilkin,6,Rank 1,6d4,25.93,23.56,23.25,2022-03-29 13:10:47
...,...,...,...,...,...,...,...,...,...
4995,Balor,Demonic,3,Rank 3,3d8+4,24.08,27.23,25.43,2022-03-29 13:10:47
4996,Black Drake,Dragon,9,Rank 1,9d4+2,34.67,37.03,35.67,2022-03-29 13:10:47
4997,Ice Mephit,Elemental,6,Rank 2,6d6,37.54,35.40,34.11,2022-03-29 13:10:47
4998,Revenant,Undead,3,Rank 0,3d2+1,6.65,5.19,5.83,2022-03-29 13:10:47


In [88]:
target = "Rarity"
features = ["Level", "Health", "Energy", "Sanity"]

## Hyperparameter Tuning

In [83]:
X_train, X_test, y_train, y_test = train_test_split(
    df[features],
    df[target],
    test_size=0.20,
    random_state=42,
    stratify=df[target],
)

param_dist = {
    "criterion": ("gini", "entropy"),
    "max_depth": (3, 6, 9),
    "max_features": (2, 3, 4)
}

n_iter = 1
for arr in param_dist.values():
    n_iter *= len(arr)

search = RandomizedSearchCV(
    RandomForestClassifier(
        n_estimators=333,
        random_state=42,
    ),
    param_distributions=param_dist,
    n_iter=n_iter,
    n_jobs=-1,
    cv=7,
    random_state=42,
)
search.fit(X_train, y_train)

RandomizedSearchCV(cv=7,
                   estimator=RandomForestClassifier(n_estimators=333,
                                                    random_state=42),
                   n_iter=18, n_jobs=-1,
                   param_distributions={'criterion': ('gini', 'entropy'),
                                        'max_depth': (3, 6, 9),
                                        'max_features': (2, 3, 4)},
                   random_state=42)

In [84]:
search.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=9, max_features=3,
                       n_estimators=333, random_state=42)

In [87]:
search.best_score_

0.9902488054852326

## Model Interface: RandomForestClassifier

### Abstraction, Encapsulation, Polymorphism

Below is one example of an abstraction that encapsulates an ML model and extends some customization points. Here we'll use a class interface, but functions can work too.

You can parameterize every aspect of the model by adding arguments to the init method. Be mindful, you don't want to over-do it here. Keep your calling signature simple and usable. Provide good defaults and well named arguments, and your users will enjoy using your code. Make it super complicated, and they may as well just use Scikit themselves.

A good interface should always encapsulate the core logic in such a way that the rest of the app is totally unaware of how it works, but can still interact with it. One might say that a good interface is always more abstract than the core logic it encapsulates. At this higher abstraction level it becomes easier to replace our core logic without disrupting other parts of the app.

...and now a word from our sponsor, Polymorphism!

One hypothetical example of Polymorphism is if we designed more than one ML model, possibly with two different ML libraries. Then gave them compatible interfaces. This gives us the ability to trade one model library for another without rewriting the whole app. We could do that at any time during development without disrupting anything. A Polymorphic system is built to be modular from the start.

In [90]:
class Model:

    def __init__(self, df: DataFrame, target: str, features: List[str]):
        X_train, X_test, y_train, y_test = train_test_split(
            df[features],
            df[target],
            test_size=0.20,
            random_state=42,
            stratify=df[target],
        )
        self.model = RandomForestClassifier(
            criterion="entropy",
            max_depth=9,
            max_features=3,
            n_jobs=-1,
            random_state=42,
        )
        self.model.fit(X_train, y_train)
        self.baseline_score = 1 / df[target].unique().shape[0]
        self.test_score = self.model.score(X_test, y_test)

    def __call__(self, feature_basis: DataFrame) -> List[Tuple]:
        prediction = self.model.predict(feature_basis)
        probability = self.model.predict_proba(feature_basis)
        return list(zip(prediction, map(max, probability)))

    def __repr__(self):
        output = (
            "Model: Random Forest Classifier",
            f"Baseline Score: {self.baseline_score:.1%}",
            f"Testing Score: {self.test_score:.1%}",
        )
        return "\n".join(output)

    def __str__(self):
        return repr(self)

    def save(self, filepath):
        joblib.dump(self, filepath)

    @staticmethod
    def open(filepath: str):
        return joblib.load(filepath)

In [91]:
model = Model(df=df, target=target, features=features)
model

Model: Random Forest Classifier
Baseline Score: 16.7%
Testing Score: 99.3%

In [92]:
model.model

RandomForestClassifier(criterion='entropy', max_depth=9, max_features=3,
                       n_jobs=-1, random_state=42)

### Make Predictions

In [81]:
test_cases = DataFrame([
    {
        "level": 1,
        "health": 5,
        "energy": 5,
        "sanity": 5,
    },
    {
        "level": 10,
        "health": 75,
        "energy": 75,
        "sanity": 75,
    },
])

In [82]:
model(test_cases)

[('Rank 1', 0.68), ('Rank 3', 0.83)]